# Continuous Genetic Algorithm From Scratch With Python

Code created by Cinthia Ungefehr based on tutorial by Cahit Bartu Yazıcı available on [Towards Data Science](https://towardsdatascience.com/continuous-genetic-algorithm-from-scratch-with-python-ff29deedd099)

## Imports

In [203]:
import numpy as np
import random as rand

## Initial Population

First let's create a function that creates an individual. Each individual is defined by a number *n_genes* of genes, each gene is a random selected value 
between the *lower_limit* and the *upper_limit*.

In [204]:
def create_individual(n_genes, upper_limit, lower_limit):
  return [round(rand.random() * (upper_limit - lower_limit) + lower_limit, 1) for x in range(n_genes)]

Now, let's create a function tha creates a population. A population is a group of indivuals, so let's do that.

In [205]:
def create_population(n_individuals, n_genes, upper_limit, lower_limit):
  return [create_individual(n_genes, upper_limit, lower_limit) for x in range(n_individuals)]

## Fitness Calculation

The Fitness calculation function calculates the fitness value of an individual.
For simplicity, let's define the fitness calculation as the sum of the genes of the individual.

In [206]:
def fit_with_gene_sum(individual):
  return sum(individual)

Obs:
> Fitness function can be calculated for **multiple parameters**. <br> For multiple parameters, **normalizing** the different parameters is very important, the difference in magnitude between different parameters may cause one of the parameters to become **obsolete** for the fitness function value. <br> Parameters can be optimized with different methods, one of the normalization methods is *rescaling*.

Obs':
> After the parameters are normalized, the importance of the parameters are determined by the **biases** given to each parameter in the fitness function. <br> Sum of the biases given to the parameters should be **1**.

## Selection

The Selection function takes the population of candidate solutions and their fitness values (a generation) and outputs the individuals that are going to be moving on to the next generation.

**Elitism** can be introduced to the genetic algorithm, which will automatically select the best individual in a generation, so we do not lose the best solution.

* Roulette wheel selection : each individual has a chance to be selected. The chance of an individual to be selected is based on the fitness value of the individual. Fitter individuals have a higher chance to be selected.

In [207]:
def roulette(individuals_sum, chance):
  individuals = list(individuals_sum.copy())
  individuals.append(chance)
  individuals = sorted(individuals)
  return individuals(chance)

* Fittest half selection : the fittest half of the candidate solutions are selected to move to the next generation.
* Random Selection : the individuals are selected randomly.

Now, let's create the selection function

In [208]:
def selection(generation, method = "Fittest Half"):
  generation["Normalized Fitness"] = sorted([generation["Fitness"][x] / sum(generation["Fitness"]) for x in range(len(generation["Fitness"]))], reverse = True)
  generation["Cumulative Sum"] = np.array(generation["Normalized Fitness"]).cumsum()

  if method == "Roulett Wheel":
    selected = []
    for x in range(len(generation["Individuals"])//2):
      selected.append(roulette(generation["Cumulative Sum"], rand.random()))
      while len(set(selected)) != len(selected):
        selected[x]= roulette(generation["Cumulative Sum"], rand.random())
    selected_individuals = [generation["Individuals"][int(selected[x])] for x in range(len(generation["Individuals"]) // 2)]
    selected_fitnesses = [generation["Fitness"][int(selected[x])] for x in range(len(generatio["Individuals"]) // 2)]
    selected = {
      "Individuals": selected_individuals,
      "Fitness": selected_fitnesses
    }

  elif method == "Fittest Half":
    selected_individuals = [generation["Individuals"][-x - 1] for x in range(int(len(generation["Individuals"]) // 2))]
    selected_fitnesses = [generation["Fitness"][-x - 1] for x in range(int(len(generation["Individuals"]) // 2))]
    selected = {
      "Individuals": selected_individuals,
      "Fitness": selected_fitnesses
    }
  
  elif method == "Random":
    selected_individuals = [generation["Individuals"][np.random.randint(1, len(generation["Fitness"]))] for x in range(int(len(generation["Individuals"]) // 2))]
    selected_fitnesses = [generation["Fitness"][-x - 1] for x in range(int(len(generation["Individuals"]) // 2))]
    selected = {
      "Individuals": selected_individuals,
      "Fitness": selected_fitnesses
    }
  return selected

## Pairing

Pairing and mating are used as a single operation in most genetic algorithm applications, but for creating simpler functions and to be able to used different mating and paring algorithms easily, the two genetic operations are separated in this application. If there is elitism in the genetic algorithm, the elit must be an input to the function as well as the selected individuals.

* Fittest: the individuals are paired two by two, starting from the fittest individual.
* Random: the individuals are paired randomly.
* Wieghted Random: the individuals are paired randomly, but fitter individuals have higher chance of been paired together.



In [209]:
def pairing(elit, selected, method = "Fittest"):
  individuals = [elit["Individuals"]] + selected["Individuals"]
  fitness = [elit["Fitness"]] + selected["Fitness"]

  if method == "Fittest":
    parents = [[individuals[x], individuals[x + 1]] for x in range(len(individuals) // 2)]
  elif method == "Random":
    parents = []
    for x in range(len(individuals) // 2):
      parents.append([
        individuals[rand.randint(0, (len(individuals) - 1))],
        individuals[rand.randint(0, len(individuals) - 1)]
      ])
      while parents[x][0] == parents[x][1]:
        parents[x][1] = individuals[rand.randint(0, len(individuals) - 1)]
  elif method == "Weighted Random":
    normalized_fitness = sorted(
      [fitness[x] / sum(fitness) for x in range(len(individuals) // 2)],
      reverse = True
    )
    cumulative_sum = np.array(normalized_fitness).cumsum()
    parents = []
    for x in range(len(individuals) // 2):
      parents.append([
        individuals[roulette(cumulative_sum, rand.random())],
        individuals[roulette(cumulative_sum, rand.random())]
      ])
      while parents[x][0] == parents[x][1]:
        parents[x][1] = individuals[roulette(cumulative_sum, rand.random())]
  return parents

## Mating

For this problem, each pair of individuals will create two offsprings.

* Single Point: the genes after a single point are replaced whith genes os the other parent to create two offsprings.
* Two Points: the genes between two point are replaced with the genes of the other parent to create two offsprings.

In [210]:
def mating(parents, method = "Single Point"):
  if method == "Single Point":
    pivot = rand.randint(1, len(parents[0]))
    offsprings = [parents[0][0 : pivot] + parents[1][pivot : ]]
    offsprings.append(parents[1][0 : pivot] + parents[0][pivot : ])
  elif method == "Two Points":
    pivot1 = rand.randint(1, len(parents[0] - 1))
    pivot2 = rand.randint(1, len(parents[0]))
    while pivot2 < pivot1:
      pivot2 = rand.randint(1, len(parents[0]))
    offsprings = [parents[0][0 : pivot1] + parents[1][pivot1 : pivot2] + parents[0][pivot2 : ]]
    offsprings.append(offsprings = [parents[1][0 : pivot1] + parents[0][pivot1 : pivot2] + parents[1][pivot2 : ]])
  return offsprings

## Mutations

Random mutations occur in the selected individuals and their offsprings to improve variety of the next generation. If there is elitism in the genetic algorithm, elit individual does not go through random mutations so we do not lose the best solution.

* Gauss: the gene that goes through mutation is replaced with a number that is generated according to Gauss Distribution around the original gene.
* Reset: the original gene is rreplaced with a randomly generated gene.

In [211]:
def mutation(individual, upper_limit, lower_limit, mutation_rate = 2, method = "Reset", standard_deviation = 0.001):
  genes = [rand.randint(0, 7)]
  for x in range(mutation_rate - 1):
    genes.append(rand.randint(0, 7))
    while len(set(genes)) < len(genes):
      genes[x] = rand.randint(0, 7)
  mutated_individual = individual.copy()
  if method == "Gauss":
    for x in range(mutation_rate):
      mutated_individual[x] = round(individual[x] + rand.gauss(0, standard_deviation), 1)
  elif method == "Reset":
    for x in range(mutation_rate):
      mutated_individual[x] = round(rand.random() * (upper_limit - lower_limit) + lower_limit, 1)
  return mutated_individual

## Next Generation

The next generation is created using the genetic operations we discussed. Elitism can be introduced to the genetic algorithm during the creating of next generation.

In [212]:
def next_generation(prev_gen, upper_limit, lower_limit):
  elit = {}
  next_gen = {}
  elit["Individuals"] = prev_gen["Individuals"].pop(-1)
  elit["Fitness"] = prev_gen["Fitness"].pop(-1)
  selected = selection(prev_gen)
  parents = pairing(elit, selected)

  offsprings = [[[mating(parents[x]) for x in range(len(parents))][y][z] for z in range(2)] for y in range(len(parents))]
  offsprings1 = [offsprings[x][0] for x in range(len(parents))]
  offsprings2 = [offsprings[x][1] for x in range(len(parents))]

  unmutated = selected["Individuals"] + offsprings1 + offsprings2
  mutated = [mutation(unmutated[x], upper_limit, lower_limit) for x in range(len(prev_gen["Individuals"]))]
  
  unsorted_individuals = mutated + [elit["Individuals"]]
  unsorted_next_gen = [fit_with_gene_sum(mutated[x]) for x in range(len(mutated))]
  unsorted_fitness = [unsorted_next_gen[x] for x in range(len(prev_gen["Fitness"]))] + [elit["Fitness"]]
  sorted_next_gen = sorted(
    [[unsorted_individuals[x], unsorted_fitness[x]] for x in range(len(unsorted_individuals))], 
    key = lambda x: x[1]
  )
  
  next_gen["Individuals"] = [sorted_next_gen[x][0] for x in range(len(sorted_next_gen))]
  next_gen["Fitness"] = [sorted_next_gen[x][1] for x in range(len(sorted_next_gen))]

  prev_gen["Individuals"].append(elit["Individuals"])
  prev_gen["Fitness"].append(elit["Fitness"])
  return next_gen


## Termination Criteria

After a generation is created, termination criteria are used to determine if the genetic algorithm should create another generation or should it stop. Different termination criteria can be used at the same time and if the genetic algorithm satisfies one of the criteria the genetic algorithm stops.

* Maximum Fitness : checks if the fittest individual in the current generation satisfies our criteria. Using this termiantion method, desired results can be obtained.
* Maximum Average Fitness: the average values of the individuals in the current generations can be checked to determine if the current generation satisfies our expectations.
* Maximum Number of Generations: limit the maximum number of generations created by the genetic algorithm.
* Maximum Similar Fitness Number: Due to elitism best individual in a generation moves on to the next generation without mutating. This individual can be the best individual in the next generation as well. We can limit the number for the same individual to be the best individual as this can be sing that the genetic algorithm got stuck in a local minima.


In [213]:
def fitness_similarity_chech(max_fitness, number_of_similarity):
  similarity = 0
  for n in range(len(max_fitness) - 1):
    if max_fitness[n] == max_fitness[n + 1]:
      similarity += 1
    else:
      similarity = 0
  return (similarity == number_of_similarity - 1)

## Running the Algorithm

In [223]:
# Generations and fitness values will be written to this file
result_file = 'GA_Results.txt'

# Creating the First Generation
def first_generation(pop):
  fitness = [fit_with_gene_sum(pop[x]) for x in range(len(pop))]
  sorted_fitness = sorted([[pop[x], fitness[x]] for x in range(len(pop))], key = lambda x: x[1])
  population = [sorted_fitness[x][0] for x in range(len(sorted_fitness))]
  fitness = [sorted_fitness[x][1] for x in range(len(sorted_fitness))]
  return {'Individuals': population, 'Fitness': sorted(fitness)}

pop = create_population(20, 8, 1, 0)
gen_iteration = 0
gen = []
gen.append(first_generation(pop))

fitness_avg = np.array([sum(gen[0]['Fitness']) / len(gen[0]['Fitness'])])
fitness_max = np.array([max(gen[0]['Fitness'])])

res = open(result_file, 'w')
res.write('\n' + str(gen) + '\n')
res.close()

while max(fitness_max) < 7 and max(fitness_avg) < 6 and not fitness_similarity_chech(fitness_max, 50):
  gen_iteration += 1

  gen.append(next_generation(gen[-1], 1, 0))
  fitness_avg = np.append(fitness_avg, sum(gen[-1]['Fitness']) / len(gen[-1]['Fitness']))
  fitness_max = np.append(fitness_max, max(gen[-1]['Fitness']))

  res = open(result_file, 'a')
  res.write('\n' + str(gen[-1]) + '\n')
  res.close()

res = open(result_file, 'r')
print(res.read())  
res.close()


[{'Individuals': [[0.4, 0.5, 0.2, 0.3, 0.4, 0.4, 0.0, 0.1], [0.1, 0.2, 0.1, 0.5, 0.2, 0.2, 0.5, 0.8], [0.2, 0.7, 0.5, 0.3, 0.4, 0.1, 0.1, 0.5], [0.2, 0.9, 0.5, 0.9, 0.1, 0.1, 0.0, 0.1], [0.4, 0.1, 0.5, 0.0, 0.0, 0.5, 0.9, 0.9], [0.3, 0.4, 0.0, 0.3, 0.0, 0.8, 0.7, 0.9], [0.7, 0.2, 0.1, 0.2, 0.1, 0.9, 0.3, 0.9], [0.9, 0.4, 0.9, 0.8, 0.0, 0.2, 0.1, 0.1], [0.9, 0.3, 0.7, 0.4, 0.7, 0.2, 0.0, 0.4], [0.5, 0.9, 1.0, 0.1, 0.6, 0.1, 0.4, 0.7], [0.5, 0.8, 0.2, 0.5, 0.7, 0.2, 0.9, 0.6], [0.2, 0.9, 0.9, 0.4, 0.6, 0.4, 0.4, 0.8], [0.3, 0.8, 0.3, 0.9, 1.0, 0.5, 0.8, 0.2], [0.2, 1.0, 0.5, 0.8, 0.7, 0.8, 0.6, 0.3], [0.7, 0.1, 0.7, 0.8, 1.0, 0.4, 0.6, 0.7], [0.8, 0.3, 0.6, 0.2, 0.8, 0.5, 0.8, 1.0], [0.9, 0.9, 0.7, 0.9, 0.8, 0.0, 0.6, 0.3], [0.7, 0.4, 0.9, 0.9, 0.5, 0.1, 0.7, 0.9], [0.7, 0.9, 0.5, 0.5, 0.6, 0.9, 0.5, 0.6], [0.8, 0.8, 0.3, 0.8, 0.2, 1.0, 0.7, 0.8]], 'Fitness': [2.3000000000000003, 2.6, 2.8000000000000003, 2.8000000000000003, 3.3, 3.4, 3.4, 3.4000000000000004, 3.6, 4.3, 4.4, 4.6, 4.800000